# 13-9. 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.<br>
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.<br>
- 별점을 시청횟수로 해석해서 생각하겠습니다.<br>
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.<br>



In [192]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.19.5
1.9.3
0.4.8


# 1) 데이터 준비와 전처리
---
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [193]:
import os
import pandas as pd
rating_file_path='./data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [194]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [195]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [196]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [197]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='./data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

# 2) 분석해 봅시다.
---
- ratings에 있는 유니크한 영화 개수<br>
- ratings에 있는 유니크한 사용자 수<br>
- 가장 인기 있는 영화 30개(인기순)<br>

In [198]:
# ratings에 movie_id로 존재하는 값과 movies에 movie_id로 존재하는 값이 같은 경우 movie의 title 컬럼을 만들고 값을 넣어주려고합니다
ratings = pd.merge(ratings,movies[['movie_id','title']],on='movie_id',how='left')
ratings

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


In [199]:
ratings['user_id'] = ratings['user_id'].astype(str)

In [200]:
# ratings에 있는 유니크한 영화개수
ratings['movie_id'].nunique()

3628

In [201]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [202]:
# 가장 인기 있느 영화 찾아보자 30개
movie_id_user_count = ratings.groupby('movie_id')['counts'].count()
best_movie_id = movie_id_user_count.sort_values(ascending=False) # 큰값이 맨처음 ---> 나중으로 갈수록 값이 작아지도록 정렬
best_movie_id_list=best_movie_id[:30].index.to_list() # 최상위 30개의 index를 list로 변환

In [203]:
best_movie_top30_df = movies[movies['movie_id'].isin(best_movie_id_list)]
best_movie_top30_df=best_movie_top30_df.set_index('movie_id') # 인덱스를 movie_id로 변경한다
best_movie_top30_df.loc[best_movie_id_list] # best_movie_id_list 값 순서와 동일하게 배치된 DataFrame을 출력한다 맨위부터 제일 많이 선호하는영화

,title,genre
movie_id,,
2858,American Beauty (1999),Comedy|Drama
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2028,Saving Private Ryan (1998),Action|Drama|War
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
593,"Silence of the Lambs, The (1991)",Drama|Thriller
1198,Raiders of the Lost Ark (1981),Action|Adventure
1270,Back to the Future (1985),Comedy|Sci-Fi


# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
---

In [204]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite_movie = ['Contender, The (2000)' , 'Tigerland (2000)' ,'Requiem for a Dream (2000)' ,'Get Carter (1971)' ,'Bootmen (2000)']

# 위에서 모든 id를 str으로 변환했기 때문에 저의 'id'는 liky 로 하겠습니다.
# id=liky 가 favorite_movie 를 보고 추천을 3으로 주었다고 가정하겠습니다.
my_rate_list = pd.DataFrame({'user_id': ['liky']*5, 'counts':[3]*5, 'timestamp':0,'title':my_favorite_movie})

if not ratings.isin({'user_id':['liky']})['user_id'].any():  # user_id에 liky 이라는 데이터가 없다면
    ratings = ratings.append(my_rate_list)                 # 위에 임의로 만든 my_rate_list 데이터를 추가해 줍니다. 

ratings.tail(10) 

,user_id,movie_id,counts,timestamp,title
836473,6040,1090.0,3,956715518,Platoon (1986)
836474,6040,1094.0,5,956704887,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569,E.T. the Extra-Terrestrial (1982)
0,liky,NaN,3,0,"Contender, The (2000)"
1,liky,NaN,3,0,Tigerland (2000)
2,liky,NaN,3,0,Requiem for a Dream (2000)
3,liky,NaN,3,0,Get Carter (1971)
4,liky,NaN,3,0,Bootmen (2000)


In [206]:
user_id = ratings['user_id'].unique()
movie_id = ratings['title'].unique()
user_to_idx = {v:k for k,v in enumerate(user_id)}
movie_to_idx = {v:k for k,v in enumerate(movie_id)}

In [207]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx 을 통해  title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

user_id column indexing OK!!
title column indexing OK!!


In [208]:
ratings.drop('movie_id',axis=1,inplace=True)

In [209]:
ratings.drop('timestamp',axis=1,inplace=True)

In [210]:
ratings.rename(columns={'title':'movie_id'},inplace=True)

In [213]:
ratings

,user_id,counts,movie_id
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
0,6039,3,1099
1,6039,3,2398
2,6039,3,1318
3,6039,3,2691


# 4) CSR matrix를 직접 만들어 봅시다.
---

In [214]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련.
---

In [271]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, use_gpu=False, iterations=200, dtype=np.float32)

In [272]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [273]:
# 모델 훈련
als_model.fit(csr_data_transpose)

100%|██████████| 200/200 [02:50<00:00,  1.17it/s]


In [274]:
als_model.item_factors.shape

(3628, 500)

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [275]:
# 내가선호하는 5가지 영화중 하나
my_user_idx, requiem_movie_idx = user_to_idx['liky'], movie_to_idx['Requiem for a Dream (2000)']

In [276]:
my_vector = als_model.user_factors[my_user_idx]
requiem_vector = als_model.item_factors[requiem_movie_idx]

In [277]:
# 선호도가 0.56으로 나오네요
np.dot(my_vector, requiem_vector)

0.56579596

In [278]:
# 제가 선택하지 않은 영화 
ab_movie_idx = movie_to_idx['American Beauty (1999)']
ab_vector = als_model.item_factors[ab_movie_idx]

In [279]:
# 선호도가 더 낮게 나오네요^^
np.dot(my_vector, ab_vector)

-0.008366175

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [280]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
def get_similar_movie(movie_name: str):
    movie_idx = movie_to_idx[movie_name]
    similar_movies = als_model.similar_items(movie_idx)
    similar_movie_list = [idx_to_movie[i[0]] for i in similar_movies]
    return similar_movie_list

In [281]:
get_similar_movie('Tigerland (2000)')

['Tigerland (2000)',
 'Two Family House (2000)',
 'Yards, The (1999)',
 'Splendor (1999)',
 'Go Now (1995)',
 'Secret Agent, The (1996)',
 'All Things Fair (1996)',
 'Hangmen Also Die (1943)',
 'Jeanne and the Perfect Guy (Jeanne et le garçon formidable) (1998)',
 "Brother's Kiss, A (1997)"]

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [282]:
movie_recommended = als_model.recommend(my_user_idx, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(755, 0.26510483),
 (1852, 0.25399625),
 (1853, 0.24022226),
 (1814, 0.19178516),
 (1887, 0.15767696),
 (2243, 0.14495523),
 (2537, 0.13773964),
 (432, 0.12729578),
 (461, 0.116106376),
 (904, 0.112489946),
 (2056, 0.111758575),
 (1786, 0.109740354),
 (374, 0.10932307),
 (208, 0.10883531),
 (1470, 0.103501976),
 (463, 0.101000056),
 (667, 0.10018546),
 (355, 0.099741675),
 (1317, 0.09955253),
 (1808, 0.09904888)]

In [283]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Almost Famous (2000)',
 'Dancer in the Dark (2000)',
 'Best in Show (2000)',
 'Nurse Betty (2000)',
 'Wonder Boys (2000)',
 'Yards, The (1999)',
 'Girlfight (2000)',
 'Taxi Driver (1976)',
 'Meet the Parents (2000)',
 'Clockwork Orange, A (1971)',
 'Way of the Gun, The (2000)',
 "Jesus' Son (1999)",
 'Exorcist, The (1973)',
 'Donnie Brasco (1997)',
 'Tao of Steve, The (2000)',
 'Frequency (2000)',
 'Fletch (1985)',
 'Straight Story, The (1999)',
 'Bamboozled (2000)',
 'Chuck & Buck (2000)']